### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [16]:
import os

from langchain.agents import AgentExecutor, create_tool_calling_agent

from langchain.tools import BaseTool, StructuredTool, tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

#from langchain_community.llms.oci_generative_ai import OCIGenAI
from langchain_community.chat_models.oci_generative_ai import ChatOCIGenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, ToolMessage

from langchain.tools import BaseTool, StructuredTool, Tool, tool
import re

from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder #ChatPromptTemplate

import pandas as pd

In [17]:
compartment_id = os.environ["NB_SESSION_COMPARTMENT_OCID"]

## ChatOCIGenAI

In [14]:
#Ref: https://python.langchain.com/v0.2/docs/integrations/chat/oci_generative_ai/
command_r_plus = "ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceya7ozidbukxwtun4ocm4ngco2jukoaht5mygpgr6gq2lgq"
command_r_plus_tag = "cohere.command-r-plus"
command_r_plus_tag2 ="cohere.command-r-plus v1.2"

In [18]:
chat = ChatOCIGenAI(
    model_id=command_r_plus,#"cohere.command-r-plus",
    provider="cohere",
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    compartment_id=compartment_id,
    model_kwargs={"temperature": 0.7, "max_tokens": 500},
)

In [119]:
df = pd.read_csv('Tabela_Futebol_Agents.csv')
    
time = 'Sport'
lista_time_A = df['Time_A'].unique()
lista_time_B = df['Time_B'].unique()

if time in lista_time_A:
    time_A = time
    time_B = df.loc[df['Time_A'] == time, 'Time_B'].iloc[0]
    gol_A = df.loc[df['Time_A'] == time, 'Gol_TimeA'].iloc[0]
    gol_B = df.loc[df['Time_A'] == time, 'Gol_TimeB'].iloc[0]

    result = f"{time_A} {gol_A} x {gol_B} {time_B}"

elif time in lista_time_B:
    time_B = time
    time_A = df.loc[df['Time_B'] == time, 'Time_A'].iloc[0]
    gol_A = df.loc[df['Time_B'] == time, 'Gol_TimeA'].iloc[0]
    gol_B = df.loc[df['Time_B'] == time, 'Gol_TimeB'].iloc[0]

    result = f"{time_A} {gol_A} x {gol_B} {time_B}"
    
print(result)


Bahia 2 x 2 Sport


In [120]:
# Contexto global para armazenar o resultado
context = {"last_result": None}

def checa_rodada(inputs: int) -> str:
    """Checa os resultados a partir do número da rodada representada por um número inteiro"""
  
    df = pd.read_csv('Tabela_Futebol_Agents.csv')
    
    rodada = df[df['Rodada'] == int(inputs)]
    
    time_A = rodada['Time_A'].astype(str)
    time_B = rodada['Time_B'].astype(str)
    
    gol_A = rodada['Gol_TimeA'].astype(str)
    gol_B = rodada['Gol_TimeB'].astype(str)
    
    result = f"{time_A.iloc[0]} {gol_A.iloc[0]} x {gol_B.iloc[0]} {time_B.iloc[0]}"
    
    return result

def checa_times(inputs: str) -> str:
    """Checa os resultados a partir de 1 dos times que jogaram. Os nomes sempre começam com letra maiúscula.
    Alguns exemplos de nomes de times: Vasco, São Paulo, Corinthians, Sport.
    """
    
    df = pd.read_csv('Tabela_Futebol_Agents.csv')
    
    time = inputs
    lista_time_A = df['Time_A'].unique()
    lista_time_B = df['Time_B'].unique()

    if time in lista_time_A:
        time_A = time
        time_B = df.loc[df['Time_A'] == time, 'Time_B'].iloc[0]
        gol_A = df.loc[df['Time_A'] == time, 'Gol_TimeA'].iloc[0]
        gol_B = df.loc[df['Time_A'] == time, 'Gol_TimeB'].iloc[0]

        result = f"{time_A} {gol_A} x {gol_B} {time_B}"

    elif time in lista_time_B:
        time_B = time
        time_A = df.loc[df['Time_B'] == time, 'Time_A'].iloc[0]
        gol_A = df.loc[df['Time_B'] == time, 'Gol_TimeA'].iloc[0]
        gol_B = df.loc[df['Time_B'] == time, 'Gol_TimeB'].iloc[0]

        result = f"{time_A} {gol_A} x {gol_B} {time_B}"

    return result


In [121]:
# Definindo as ferramentas
description_rodada = ''' 
A partir do número da rodada, checa os resultados dos dois times que disputaram esta rodada.
'''
description_team = ''' 
A partir do nome de um time, checa os resultados da rodada que este time participou.
'''

round_tool = Tool.from_function(checa_rodada, name="checa_rodada", description=description_rodada)
team_tool = Tool.from_function(checa_times, name="checa_time", description=description_team)

tools_list = [round_tool, team_tool]

In [122]:
# llm with tools
llm_with_tools = chat.bind_tools(tools_list)

In [129]:
# Prompt para criar o Agente de Chamadas de Ferramentas
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um assistente digital que responde perguntas sobre resultados de jogos de futebol"),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construção do Agente de Chamadas de Ferramentas com as ferramentas personalizadas
agent = create_tool_calling_agent(llm_with_tools, tools_list, prompt)

# Criação do executor do agente passando o agente e as ferramentas
agent_executor = AgentExecutor(agent=agent, tools=tools_list, verbose=True, max_iterations=100)#, early_stopping_method="force")

# Executando o Agente com uma pergunta relacionada a operações matemáticas
query = "Qual o resultado do jogo do Vasco?"
result = agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...

Invoking: `checa_time` with `{'tool_input': 'Vasco'}`
responded: Vou procurar pelo último jogo do Vasco.Vou procurar pelo último jogo do Vasco.Vou procurar pelo último jogo do Vasco.

Vasco 0 x 2 FlamengoO Vasco perdeu de 2x0 para o Flamengo.

> Finished chain.


In [131]:
print(result['output'])

O Vasco perdeu de 2x0 para o Flamengo.
